In [1]:
import cmdstanpy
import numpy as np
import pandas as pd

from IPython.display import clear_output
from itertools import product
cmdstanpy.install_cmdstan();

CmdStan install directory: /Users/igor.michels/.cmdstan
CmdStan version 2.34.1 already installed
Test model compilation


In [2]:
NUM_SEASONS = 1
NUM_TEAMS = 20

In [3]:
def generate_matchups(n_equipes, habilidades, home_force):
    if home_force is None: home_force = 0
    confrontos = []
    for equipe1, equipe2 in product(range(1, n_equipes + 1), repeat=2):
        if equipe1 == equipe2: continue
        prob = np.exp(habilidades[equipe1 - 1] + home_force)
        prob = prob / (np.exp(habilidades[equipe1 - 1] + home_force) + np.exp(habilidades[equipe2 - 1]))
        confrontos.append({
            'equipe1': equipe1,
            'equipe2': equipe2,
            'vitoria_equipe1': (np.random.random() < prob) * 1
        })

    df_confrontos = pd.DataFrame(confrontos)

    return df_confrontos

def generate_seasons(n_equipes, habilidades, n_seasons, home_force=None):
    df = pd.DataFrame()
    for _ in range(n_seasons):
        df = pd.concat([df, generate_matchups(n_equipes, habilidades, home_force)])
    
    return df

In [4]:
habilidades = np.random.normal(0, 1, NUM_TEAMS)
df = generate_seasons(NUM_TEAMS, habilidades, NUM_SEASONS)

modelo_compilado = cmdstanpy.CmdStanModel(stan_file='../models/BT_model.stan', force_compile=False)
dados_modelo = {
    'num_jogos': len(df),
    'num_equipes': NUM_TEAMS,
    'equipe1': df['equipe1'].values,
    'equipe2': df['equipe2'].values,
    'vitoria_equipe1': df['vitoria_equipe1'].values,
}

ajuste = modelo_compilado.sample(data=dados_modelo, chains=4, iter_warmup=2000, iter_sampling=10000, show_console=False)
habilidades += ajuste.summary()['Mean']['habilidade[1]'] - habilidades[0]
inside_ci = (habilidades > ajuste.summary()['5%'].values[1:])
inside_ci *= ((habilidades < ajuste.summary()['95%'].values[1:]))

clear_output()
sum(inside_ci) / NUM_TEAMS

0.8

In [5]:
home_force = abs(np.random.normal(0, 1))
habilidades = np.random.normal(0, 1, NUM_TEAMS)
df = generate_seasons(NUM_TEAMS, habilidades, NUM_SEASONS, home_force)

modelo_compilado = cmdstanpy.CmdStanModel(stan_file='../models/BT_model_2.stan', force_compile=False)
dados_modelo = {
    'num_jogos': len(df),
    'num_equipes': NUM_TEAMS,
    'equipe1': df['equipe1'].values,
    'equipe2': df['equipe2'].values,
    'vitoria_equipe1': df['vitoria_equipe1'].values,
}

ajuste = modelo_compilado.sample(data=dados_modelo, chains=4, iter_warmup=2000, iter_sampling=10000, show_console=False)
habilidades += ajuste.summary()['Mean']['habilidade[1]'] - habilidades[0]
inside_ci = (habilidades > ajuste.summary()['5%'].values[2:])
inside_ci *= ((habilidades < ajuste.summary()['95%'].values[2:]))

clear_output()
sum(inside_ci) / NUM_TEAMS

1.0

In [6]:
home_force

0.7513962642717635

In [7]:
ajuste.summary()

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
lp__,5.215700e+06,3.758920e+06,5.729850e+06,1448.98000,2.137430e+06,1.359200e+07,2.32359,0.250333,4.07006
home_force,1.161260e+00,6.115270e+00,8.660280e+00,-8.17792,8.266650e+00,1.016000e+01,2.00554,0.216068,20.80210
habilidade[1],-3.956550e+00,2.972240e+00,4.226430e+00,-9.03340,-3.945920e-01,8.304630e-01,2.02199,0.217840,9.06032
habilidade[2],-6.804520e+00,5.231870e+00,7.416910e+00,-15.22880,3.434940e-01,8.327750e-01,2.00971,0.216516,13.84170
habilidade[3],-6.991610e+00,5.210030e+00,7.385640e+00,-15.20570,-1.939760e+00,2.366160e+00,2.00954,0.216498,13.96280
habilidade[4],-5.168050e+00,4.241840e+00,6.025780e+00,-12.32400,3.579920e-01,1.271500e+00,2.01798,0.217408,10.04270
habilidade[5],-6.329400e+00,4.660450e+00,6.614530e+00,-13.91280,-1.722530e+00,1.971350e+00,2.01438,0.217020,11.26470
habilidade[6],-7.120780e+00,5.024400e+00,7.123920e+00,-15.21350,-9.322000e-01,8.376230e-01,2.01034,0.216585,13.39270
habilidade[7],-4.342940e+00,3.377930e+00,4.801510e+00,-10.09010,-1.066340e-01,9.329950e-01,2.02048,0.217678,9.39363
habilidade[8],-4.294080e+00,3.039160e+00,4.322620e+00,-9.53319,-1.709650e-01,1.617590e-01,2.02296,0.217944,8.86683
